In [1]:
## added if case for handling list index out of range error

In [2]:
import time
st = time.time()

from img2table.document import Image
from img2table.ocr import PaddleOCR
import pandas as pd
from pdf2image import convert_from_path

from PIL import Image as PIL_Image, ImageDraw

In [3]:
# Dictionary for storing marks of each papers
my_dict = {'1a': [], '1b': [], '1c': [], '2a': [], '2b': [], '2c': [], '3a': [], '3b': [], '3c': [], '4a': [], '4b': [], '4c': [], '5a': [], '5b': [], '5c': [], '6a': [], '6b': [], '6c': [], '7a': [], '7b': [], '7c': [], '8a': [], '8b': [], '8c': [], '9a': [], '9b': [], '9c': [], '10a': [], '10b': [], '10c': [], '11a': [], '11b': [], '11c': [], '12a': [], '12b': [], '12c': []}

In [4]:
images = convert_from_path('D:/AJAYMON/AJAY/Programming/Auto_Excel_Mark_Entry/Codes/image to table/MP_pdf2.pdf') # change or give PDF name here....................new_exm.pdf, MP_pdf1.pdf

for i in range(len(images)):
    img = images[i]
    _, height = img.size
    cropped_img = img.crop((150, height / 2 + 50, 1600, height - 300)) # left, top, right, bottom
    cropped_img.save(f"D:/AJAYMON/AJAY/Programming/Auto_Excel_Mark_Entry/Codes/image to table/temp/ima{i}.jpg")
    
    # img = np.array(cropped_img)

    # Convert the image to grayscale
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # # Apply thresholding to make the black lines more black and white areas more white
    # thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # # Apply morphological operations to further enhance the image
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    # closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # # Apply dilation to make the lines thicker
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    # dilated = cv2.dilate(closed, kernel, iterations=1)
    
    # cv2.imwrite(f"D:/AJAYMON/AJAY/Programming/Auto_Excel_Mark_Entry/Codes/image to table/temp/ima{i}.jpg", gray)

# ----------------------------------------------------------------------------------------------------------------------------------    

In [5]:
for i in range(len(images)):
    ## Image opening & performing OCR
    paddle_ocr = PaddleOCR(lang="en")
    src = f"D:/AJAYMON/AJAY/Programming/Auto_Excel_Mark_Entry/Codes/image to table/temp/ima{i}.jpg"

    doc = Image(src, dpi=200)
    extracted_tables = doc.extract_tables(ocr=paddle_ocr, implicit_rows=True, min_confidence=50)

    # Load the image using PIL
    img = PIL_Image.open(src) # comment this --------------------------------------------------

    # Create a draw object
    draw = ImageDraw.Draw(img) # comment this --------------------------------------------------

    for table in extracted_tables:
        for row in table.content.values():
            for cell in row:
                draw.rectangle((cell.bbox.x1, cell.bbox.y1, cell.bbox.x2, cell.bbox.y2), outline="red", width=3)
                
    img.save(f"D:/AJAYMON/AJAY/Programming/Auto_Excel_Mark_Entry/Codes/image to table/temp/img{i}_with_redlines{i}.jpg") # comment this --------------------------------------------------

    df = table.df
    df = df.iloc[1:, 1:] # deleting first row and column
    df = df.drop(index=df.index[-1]) # remove the last row

    ## Flattening & adding marks to my_dict
    arr = df.to_numpy()
    flat = arr.flatten(order='F') # flattening column-wise
    cell_vals = [i for i in flat]

    flat_len = len(cell_vals)

    print(i+1, "-", flat_len) # comment this --------------------------------------------------

    if flat_len < 36:
        # my_dict = {key: None for key in my_dict}
        pass
    else:
        # Adding values to dictionary
        i = 0
        for key in my_dict:
            my_dict[key].append(cell_vals[i])
            i+=1

1 - 36
2 - 36
3 - 36
4 - 36
5 - 36
6 - 36
7 - 36
8 - 36
9 - 36
10 - 36
11 - 39
12 - 33
13 - 36
14 - 36
15 - 36
16 - 36
17 - 36
18 - 36
19 - 36
20 - 36
21 - 36
22 - 33
23 - 33
24 - 36
25 - 36
26 - 36
27 - 36
28 - 36
29 - 36
30 - 36
31 - 36
32 - 36
33 - 36
34 - 36


In [6]:
## Dictionary to dataframe & it's preprocessing

### The df output may have 'None' values but they're actually NaN, so they won't come in exported CSV file
# dict_df = pd.DataFrame(my_dict)
# col_name = dict_df.columns[(dict_df == 'None').all()] # finding the column with "None" word
# dict_df = dict_df.drop(col_name, axis=1) # delete the identified columns
# dict_df = dict_df.replace(to_replace="None", value=np.nan) # Replacing all "None" to NaN, which will be empty when converted to CSV
# dict_df = dict_df.assign(sum=dict_df.sum(axis=1)) # create a new column "row_sum" with the sum of each row
# dict_df = dict_df.assign(Sum_more_than_50=dict_df.apply(lambda x: 'Error' if x['sum'] > 50 else '', axis=1)) # add a new column "error" with "Error" if row_sum is greater than 50

# print(dict_df)

dict_df = pd.DataFrame(my_dict)

In [7]:
dict_df.to_csv("D:/AJAYMON/AJAY/Programming/Auto_Excel_Mark_Entry/Codes/image to table/unprocessed_dict.csv", index=False)


# col_name = dict_df.columns[(dict_df == 'None').all()] # finding the column with "None" word
# dict_df = dict_df.drop(col_name, axis=1) # delete the identified columns
# dict_df = dict_df.replace(to_replace="None", value=np.nan) # Replacing all "None" to NaN, which will be empty when converted to CSV
# valid_cols = dict_df.select_dtypes(include=np.number).columns # Select only the valid columns with numeric data types
# dict_df = dict_df.assign(sum=dict_df[valid_cols].sum(axis=1)) # create a new column "row_sum" with the sum of each row
# dict_df = dict_df.assign(Sum_more_than_50=dict_df.apply(lambda x: 'Error' if x['sum'] > 50 else '', axis=1)) # add a new column "error" with "Error" if row_sum is greater than 50

dict_df = dict_df.dropna(axis=1, how='all')
dict_df.fillna(0, inplace=True)

In [8]:
print(dict_df)

   1a  2a 3a 4a 5a 6a 7a 8a 9a 10a 11a 12a
0   0   0  3  0  0  4  0  0  0   0   0   0
1   3   0  3  0  0  4  -  0  0   0   0   0
2   3   3  2  0  0  5  5  0  3   0   3   2
3   3   3  a  3  a  0  0  0  3   0   0   0
4   3   3  3  0  0  0  0  0  5   S   0   0
5   2   3  3  0  0  7  0  0  9   2   0   0
6   3   3  3  3  3  0  S  0  0   7   0   6
7   3   3  3  0  0  0  0  0  0   S   0   7
8   3   2  2  0  0  2  0  0  0   0   0   0
9   3   &  3  0  0  0  0  3  0   0   0   4
10  0   &  2  0  0  0  6  0  0   0   0   4
11  0   0  2  2  0  4  0  3  2   0   0   0
12  2   0  3  3  3  0  0  7  6   0   0   0
13  3  23  3  0  0  0  0  0  0   0   0   0
14  0   0  3  0  0  0  0  0  0   3   0   3
15  3   0  3  2  2  0  7  0  S   0   2   0
16  2   3  3  0  0  0  0  a  0   0   0   0
17  3   a  3  0  0  2  2  0  0   0   0   0
18  3   &  3  0  0  2  2  0  -   0   2   0
19  3   3  3  0  0  0  0  S  4   S   0   3
20  3   2  2  3  3  0  0  0  0   7   0   0
21  2   0  3  0  0  0  S  0  0   5   0   0
22  &   3  

In [9]:
# saving dict as csv
dict_df.to_csv("D:/AJAYMON/AJAY/Programming/Auto_Excel_Mark_Entry/Codes/image to table/MP2.csv", index=False)

In [10]:
print(time.time() - st)

137.05335187911987


In [26]:
extracted_tables[0].df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Qn.\nNo.,1,2,3,4,5,6,7,8,9,10,11,12
1,a,None,3,3,None,&,6,None,None,None,6,5,None
2,b,None,None,None,None,None,None,None,None,None,None,None,None
3,c,None,None,None,None,None,None,None,None,None,None,None,None
4,Total,None,None,None,None,None,None,None,None,None,None,None,None


In [13]:
for i in extracted_tables:
    print(i.content)

ExtractedTable(title=None, bbox=(42, 94, 1298, 588),shape=(5, 13))


In [27]:
for table in extracted_tables:
    for row in table.content.values():
        for cell in row:
            print(cell)

TableCell(bbox=BBox(x1=42, y1=94, x2=138, y2=191), value='Qn.\nNo.')
TableCell(bbox=BBox(x1=138, y1=94, x2=235, y2=191), value='1')
TableCell(bbox=BBox(x1=235, y1=94, x2=331, y2=191), value='2')
TableCell(bbox=BBox(x1=331, y1=94, x2=428, y2=191), value='3')
TableCell(bbox=BBox(x1=428, y1=94, x2=525, y2=191), value='4')
TableCell(bbox=BBox(x1=525, y1=94, x2=621, y2=191), value='5')
TableCell(bbox=BBox(x1=621, y1=94, x2=717, y2=191), value='6')
TableCell(bbox=BBox(x1=717, y1=94, x2=815, y2=191), value='7')
TableCell(bbox=BBox(x1=815, y1=94, x2=912, y2=191), value='8')
TableCell(bbox=BBox(x1=912, y1=94, x2=1008, y2=191), value='9')
TableCell(bbox=BBox(x1=1008, y1=94, x2=1104, y2=191), value='10')
TableCell(bbox=BBox(x1=1104, y1=94, x2=1202, y2=191), value='11')
TableCell(bbox=BBox(x1=1202, y1=94, x2=1298, y2=191), value='12')
TableCell(bbox=BBox(x1=42, y1=191, x2=138, y2=290), value='a')
TableCell(bbox=BBox(x1=138, y1=191, x2=235, y2=290), value=None)
TableCell(bbox=BBox(x1=235, y1=191, x

In [34]:
table_values = extracted_tables[0].content.values()
table_values

odict_values([[TableCell(bbox=BBox(x1=42, y1=94, x2=138, y2=191), value='Qn.\nNo.'), TableCell(bbox=BBox(x1=138, y1=94, x2=235, y2=191), value='1'), TableCell(bbox=BBox(x1=235, y1=94, x2=331, y2=191), value='2'), TableCell(bbox=BBox(x1=331, y1=94, x2=428, y2=191), value='3'), TableCell(bbox=BBox(x1=428, y1=94, x2=525, y2=191), value='4'), TableCell(bbox=BBox(x1=525, y1=94, x2=621, y2=191), value='5'), TableCell(bbox=BBox(x1=621, y1=94, x2=717, y2=191), value='6'), TableCell(bbox=BBox(x1=717, y1=94, x2=815, y2=191), value='7'), TableCell(bbox=BBox(x1=815, y1=94, x2=912, y2=191), value='8'), TableCell(bbox=BBox(x1=912, y1=94, x2=1008, y2=191), value='9'), TableCell(bbox=BBox(x1=1008, y1=94, x2=1104, y2=191), value='10'), TableCell(bbox=BBox(x1=1104, y1=94, x2=1202, y2=191), value='11'), TableCell(bbox=BBox(x1=1202, y1=94, x2=1298, y2=191), value='12')], [TableCell(bbox=BBox(x1=42, y1=191, x2=138, y2=290), value='a'), TableCell(bbox=BBox(x1=138, y1=191, x2=235, y2=290), value=None), Table

In [38]:
for i in table_values:
    print(i[1].bbox)

BBox(x1=138, y1=94, x2=235, y2=191)
BBox(x1=138, y1=191, x2=235, y2=290)
BBox(x1=138, y1=290, x2=235, y2=389)
BBox(x1=138, y1=389, x2=235, y2=488)
BBox(x1=138, y1=488, x2=235, y2=588)
